## Imports

In [2]:
# !pip install nba_api

import requests, numpy as np, pandas as pd

pd.set_option('display.max_columns', None)

from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players 

## Player Career Stats

In [3]:
career = playercareerstats.PlayerCareerStats(player_id='1630534')
player_career = career.get_data_frames()[0]
player_career['PPG'] = round(player_career['PTS'] / player_career['GP'], 2)
display(player_career) 
players.find_players_by_full_name('Ben Wallace')
# player_career['SEASON_ID']
# playercareerstats.PlayerCareerStatsget_active_players() 


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PPG
0,1630534,2022-23,00,1610612762,UTA,23.0,59,22,1209.0,165,386,0.427,81,228,0.355,56,69,0.812,43,78,121,67,16,15,41,99,467,7.92
1,1630534,2023-24,00,1610612762,UTA,24.0,51,10,1003.0,106,249,0.426,47,142,0.331,15,20,0.750,35,91,126,47,27,29,34,66,274,5.37
2,1630534,2023-24,00,1610612761,TOR,24.0,27,18,638.0,72,184,0.391,15,69,0.217,22,36,0.611,39,51,90,36,20,15,30,51,181,6.70
3,1630534,2023-24,00,0,TOT,24.0,78,28,1642.0,178,433,0.411,62,211,0.294,37,56,0.661,74,142,216,83,47,44,64,117,455,5.83


[{'id': 1112,
  'full_name': 'Ben Wallace',
  'first_name': 'Ben',
  'last_name': 'Wallace',
  'is_active': False}]

## Player Profile

In [112]:
info = commonplayerinfo.CommonPlayerInfo(player_id='1630534')
info.get_data_frames()[0]
# info.available_seasons.get_data_frame()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,1630534,Ochai,Agbaji,Ochai Agbaji,"Agbaji, Ochai",O. Agbaji,ochai-agbaji,2000-04-20T00:00:00,Kansas,USA,...,ochai_agbaji,2022,2023,Y,Y,Y,2022,1,14,N


## Stat Functions 

In [2]:
def ppg_career(id):
    career = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0] 
    return round(np.mean(career['PTS'] / career['GP']), 2)

def ppg_2023(id): 
    career = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0] 
    s_2023 = career[career['SEASON_ID'] == '2023-24'] 
    return round(np.sum(s_2023['PTS']) / np.sum(s_2023['GP']), 2)

def rpg_2023(id):
    career = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0]
    s_2023 = career[career['SEASON_ID'] == '2023-24'] 
    return round(np.sum(s_2023['REB']) / np.sum(np.sum(s_2023['GP'])), 2)  

def get_stats(id, season, stats):
    career_stats = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0] 
    season_stats = career_stats[career_stats['SEASON_ID'] == season].tail(1)  
    
    new_stats = [] 
    
    for stat in stats:
        new_stats.append(np.sum(season_stats[stat])) 
        
    return new_stats 

# def new_stats(id_col):     


## All Players Table 

In [3]:


# All players 
players_data = pd.json_normalize(players.get_players())
players_data 

# Players 2023-2024 Season 
players_data = players_data.loc[players_data.is_active == True, ['id', 'full_name']]
players_data['stats'] = players_data['id'].apply(lambda x: get_stats(x, '2023-24', ['GP','PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF'])) 


players_data 

,id,full_name,stats
10,1630173,Precious Achiuwa,"[74, 565, 487, 97, 46, 68, 83, 143]"
22,203500,Steven Adams,"[0, 0, 0, 0, 0, 0, 0, 0]"
24,1628389,Bam Adebayo,"[71, 1367, 737, 278, 81, 66, 162, 159]"
29,1630534,Ochai Agbaji,"[78, 455, 216, 83, 47, 44, 64, 117]"
41,1630583,Santi Aldama,"[61, 654, 352, 138, 43, 54, 69, 89]"
...,...,...,...
4875,201152,Thaddeus Young,"[33, 137, 103, 57, 23, 5, 15, 49]"
4877,1629027,Trae Young,"[54, 1389, 149, 583, 72, 11, 235, 109]"
4878,1630209,Omer Yurtseven,"[48, 222, 208, 29, 8, 18, 37, 52]"
4881,203469,Cody Zeller,"[43, 76, 112, 39, 9, 5, 16, 45]"


In [5]:
players_data[['gp','pts', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']] = pd.DataFrame(players_data['stats'].tolist(), index=players_data.index)


,id,full_name,stats,gp,pts,reb,ast,stl,blk,tov,pf
10,1630173,Precious Achiuwa,"[74, 565, 487, 97, 46, 68, 83, 143]",74,565,487,97,46,68,83,143
22,203500,Steven Adams,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0
24,1628389,Bam Adebayo,"[71, 1367, 737, 278, 81, 66, 162, 159]",71,1367,737,278,81,66,162,159
29,1630534,Ochai Agbaji,"[78, 455, 216, 83, 47, 44, 64, 117]",78,455,216,83,47,44,64,117
41,1630583,Santi Aldama,"[61, 654, 352, 138, 43, 54, 69, 89]",61,654,352,138,43,54,69,89
...,...,...,...,...,...,...,...,...,...,...,...
4875,201152,Thaddeus Young,"[33, 137, 103, 57, 23, 5, 15, 49]",33,137,103,57,23,5,15,49
4877,1629027,Trae Young,"[54, 1389, 149, 583, 72, 11, 235, 109]",54,1389,149,583,72,11,235,109
4878,1630209,Omer Yurtseven,"[48, 222, 208, 29, 8, 18, 37, 52]",48,222,208,29,8,18,37,52
4881,203469,Cody Zeller,"[43, 76, 112, 39, 9, 5, 16, 45]",43,76,112,39,9,5,16,45


In [10]:

players_data[['ppg', 'rpg', 'apg', 'spg', 'bpg']] =np.round(players_data[['pts', 'reb', 'ast', 'stl', 'blk']].div(players_data['gp'], axis=0), 1)
players_data.fillna(0, inplace=True)
players_data

,id,full_name,stats,gp,pts,reb,ast,stl,blk,tov,pf,ppg,rpg,apg,spg,bpg
10,1630173,Precious Achiuwa,"[74, 565, 487, 97, 46, 68, 83, 143]",74,565,487,97,46,68,83,143,7.6,6.6,1.3,0.6,0.9
22,203500,Steven Adams,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
24,1628389,Bam Adebayo,"[71, 1367, 737, 278, 81, 66, 162, 159]",71,1367,737,278,81,66,162,159,19.3,10.4,3.9,1.1,0.9
29,1630534,Ochai Agbaji,"[78, 455, 216, 83, 47, 44, 64, 117]",78,455,216,83,47,44,64,117,5.8,2.8,1.1,0.6,0.6
41,1630583,Santi Aldama,"[61, 654, 352, 138, 43, 54, 69, 89]",61,654,352,138,43,54,69,89,10.7,5.8,2.3,0.7,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,201152,Thaddeus Young,"[33, 137, 103, 57, 23, 5, 15, 49]",33,137,103,57,23,5,15,49,4.2,3.1,1.7,0.7,0.2
4877,1629027,Trae Young,"[54, 1389, 149, 583, 72, 11, 235, 109]",54,1389,149,583,72,11,235,109,25.7,2.8,10.8,1.3,0.2
4878,1630209,Omer Yurtseven,"[48, 222, 208, 29, 8, 18, 37, 52]",48,222,208,29,8,18,37,52,4.6,4.3,0.6,0.2,0.4
4881,203469,Cody Zeller,"[43, 76, 112, 39, 9, 5, 16, 45]",43,76,112,39,9,5,16,45,1.8,2.6,0.9,0.2,0.1


## Live Score Data 

In [6]:
from nba_api.live.nba.endpoints import scoreboard 

games = scoreboard.ScoreBoard()
games.get_json()

'{"meta": {"version": 1, "request": "https://nba-prod-us-east-1-mediaops-stats.s3.amazonaws.com/NBA/liveData/scoreboard/todaysScoreboard_00.json", "time": "2024-06-16 02:31:06.316", "code": 200}, "scoreboard": {"gameDate": "2024-06-16", "leagueId": "00", "leagueName": "National Basketball Association", "games": []}}'

## Team Data

In [7]:
from nba_api.stats.static import teams 
pd.json_normalize(teams.find_teams_by_city('toronto'))

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612761,Toronto Raptors,TOR,Raptors,Toronto,Ontario,1995
